### Test the usage of random forest
Training a Random forest 
Sep 4, 2020

In [ ]:
import numpy as np
import glob
import os
import pandas as pd
import pickle
import time

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib widget

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_curve

In [ ]:
# from sklearn.metrics import plot_roc_curve

In [ ]:
# data_dir='/global/cfs/cdirs/dasrepo/vpa/supernova_cnn/data/autoscan_data/data/'

In [ ]:
fname='/global/cfs/cdirs/dasrepo/vpa/supernova_cnn/data/gathered_data/autoscan_features.3.csv'
df_full=pd.read_csv(fname,sep=',',comment='#')

### Convert BAND to integers
bm = {'g':0, 'r':1, 'i':2, 'z':3}
df_full['BAND']=np.array([bm[i] for i in df_full.BAND.values])
df_full.head()

In [ ]:
df=df_full[:].dropna()
df.shape

y_cols=['OBJECT_TYPE']
## Find columns with Nans
# df.columns[df_full.isna().any().values]
# reject_cols=['A_REF', 'B_REF', 'MAG_REF', 'MAG_REF_ERR', 'NN_DIST_RENORM']
reject_cols=[]
x_cols=[i for i in df.columns if i not in ['OBJECT_TYPE','ID']+reject_cols]


In [66]:
### Get IDs for test data
results_dir='/global/project/projectdirs/dasrepo/vpa/supernova_cnn/data/results_data/results/'
# fname='/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Supernova-cnn/cnn_supernova/4_train_random_forest/train_ids.train'
fname=results_dir+'id_train_3.train'
train_ids=np.loadtxt(fname,dtype=np.int32)[:]
fname=results_dir+'id_test_3.test'
test_ids=np.loadtxt(fname,dtype=np.int32)[:]


# Get train data
# train_size=801000
## IDs not in test ids
df_temp=df[~df_full.ID.isin(test_ids)].dropna()
# df_temp=df[df.ID.isin(train_ids)].dropna()
X_train,y_train=df_temp[x_cols],df_temp[y_cols]

# Get test data . This should be the same set used for testing the CNNs
df_temp=df_full[df_full.ID.isin(test_ids)].dropna()
X_test,y_test=df_temp[x_cols],df_temp[y_cols]
print(X_train.shape,X_test.shape)

/global/homes/v/vpa/.conda/envs/v_py3/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  del sys.path[0]


(801005, 38) (42144, 38)


In [62]:
# (df_full.shape[0]-df_full.dropna().shape[0])/df_full.shape[0]
(df_temp.shape[0]-df_temp.dropna().shape[0])/df_temp.shape[0]

0.06238319836255228

In [76]:
X_train.shape,X_test.shape

((801005, 38), (42144, 38))

In [ ]:
# df=df.dropna()

# X=df[x_cols]
# y=df[y_cols]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
# print(X_train.shape,X_test.shape)

### Test with Kfold CV

In [16]:
# X_train.reset_index(),y_train.reset_index();

# kf=StratifiedKFold(n_splits=4,shuffle=True)
# model=RandomForestClassifier(n_estimators=100, criterion='entropy', n_jobs=-1, max_features=6,
#                              max_depth=None, min_samples_split=3, verbose=1)


# fprs,tprs,scores=[],[],[]
# for train_idx,test_idx in kf.split(X_train,y_train):
#     print(train_idx.shape,test_idx.shape)
#     print(np.mean(train_idx),np.mean(test_idx))
#     model.fit(X_train.iloc[train_idx],y_train.iloc[train_idx].OBJECT_TYPE.values)
    
#     y_predict=model.predict_proba(X_train.iloc[test_idx])
#     fpr,tpr,thrs=roc_curve(y_train.iloc[test_idx],y_predict[:,1])
#     fprs.append(fpr)
#     tprs.append(tpr)
    
    

In [17]:
# plt.figure()
# for i in range(len(fprs)):
#     fpr,tpr=fprs[i],tprs[i]
#     x,y=fpr,(1-tpr)
#     plt.plot(x, y,linestyle='',markersize=2,marker='*')
#     # ### Reference points in mdr plot in paper
    
# plt.plot(0.03,0.038,marker='s',markersize=8,color='k')
# plt.plot(0.04,0.024,marker='s',markersize=8,color='k')
# plt.plot(0.05,0.016,marker='s',markersize=8,color='k')

# plt.xlabel('MDR')
# plt.ylabel('FPR')
# plt.xlim(0,0.1)
# plt.ylim(0,0.05)

### Reading feature weights in paper

In [ ]:
# ### Load importance of features 

# fle='features.txt'
# a1=np.loadtxt(fle,delimiter=',',dtype=str)

# weight_dict1={}
# for i in range(a1.shape[0]):
#     key,val=a1[i][0],a1[i][1]
#     weight_dict1[key.upper()]=val

# # [i for i in df_full.columns if i not in weight_dict.keys()],[i for i in weight_dict.keys() if i not in df.columns]
    
# key_maps={'R_APER_PSF':'GFLUX','DIFFSUM':'DIFFSUMRN','MIN_DIST_TO_EDGE':'MIN_DISTANCE_TO_EDGE_IN_NEW','NUMNEG':'NUMNEGRN','BANDNUM':'BAND'}
# # Renaming keys to match those in DataFrame
# weight_dict={}
# for key in weight_dict1.keys():
#     if key not in key_maps:
#         weight_dict[key]=float(weight_dict1[key])
#     else : 
#         weight_dict[key_maps[key]]=float(weight_dict1[key])

# # weight_dict

### Run model

In [64]:
### Train model
t1=time.time()
model=RandomForestClassifier(n_estimators=100, criterion='entropy', n_jobs=32, max_features=6,
                             max_depth=None, min_samples_split=3, verbose=1)
model.fit(X_train,y_train.OBJECT_TYPE.values)
t2=time.time()
print("Training time",t2-t1)



[Parallel(n_jobs=32)]: Using backend ThreadingBackend with 32 concurrent workers.


Training time 101.44010734558105


[Parallel(n_jobs=32)]: Done 100 out of 100 | elapsed:  1.7min finished


In [67]:
### Test model
t1=time.time()
y_predict=model.predict_proba(X_test)
t2=time.time()
print("Time of testing model",t2-t1)
fpr,tpr,thrs=roc_curve(y_test,y_predict[:,1])

[Parallel(n_jobs=32)]: Using backend ThreadingBackend with 32 concurrent workers.
[Parallel(n_jobs=32)]: Done 100 out of 100 | elapsed:    0.1s finished


Time of testing model 0.21615886688232422


In [68]:
model.score(X_test,y_test.values)

[Parallel(n_jobs=32)]: Using backend ThreadingBackend with 32 concurrent workers.
[Parallel(n_jobs=32)]: Done 100 out of 100 | elapsed:    0.1s finished


0.9685839028094153

In [31]:
fpr2,tpr2=fpr,tpr

### Plot roc curve

In [69]:
def f_roc(fpr,tpr,label):
    x,y=fpr,(1-tpr)
#     plt.figure()
    plt.plot(x, y,label=label,linestyle='',markersize=2,marker='*')
    # ### Reference points in mdr plot in paper
    plt.plot(0.03,0.038,marker='s',markersize=8,color='k')
    plt.plot(0.04,0.024,marker='s',markersize=8,color='k')
    plt.plot(0.05,0.016,marker='s',markersize=8,color='k')

    plt.xlabel('MDR')
    plt.ylabel('FPR')
    plt.xlim(0,0.1)
    plt.ylim(0,0.05)


In [70]:
plt.figure()
f_roc(fpr,tpr,'train_400k')
# f_roc(fpr2,tpr2,'train_800k')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
# for i in range(100,110):
#     print(y_test.values[i,0],y_predict[i,0],y_predict[i,1])


In [47]:
np.array_equal(X_test.AMP.values,df[df.ID.isin(test_ids)].AMP.values)

True

In [71]:
df_save=pd.DataFrame([])
df_save['ID']=df[df.ID.isin(test_ids)].ID.values
df_save['m_rf']=y_predict[:,1]

In [72]:
df_save.head()

,ID,m_rf
0,11262110,0.0
1,11741241,0.0
2,11826120,0.0
3,10093117,0.0
4,9741882,0.0


In [74]:
df_save.to_csv('rf_preds.csv',index=False)